In [ ]:
#!/usr/bin/env python  
# -*- coding: utf-8 -*-  
"""  
══════════════════════════════════════════════════════════════════════════════  
                H I T A C H I   R E B R A N D I N G   T O O L  
══════════════════════════════════════════════════════════════════════════════  
  
Replaces ONLY genuine LOGOS  
    • “Hitachi Inspire the Next”  
    • “Hitachi Energy”  (incl. red square symbol)  
  
2025-05-25  –  screenshot / text false-positive patch  
─────────────────────────────────────────────────────────────────  
Problem  
–––––––  
Even after edge-ring filtering, some occurrences of the WORDS  
“Hitachi Energy” inside normal body text or within screen-shots were  
still replaced by the logo.  
  
Root cause  
──────────  
The red-square test measured the *global* red pixel ratio of the  
whole bounding box.  A small red icon or UI element somewhere inside  
a screen-shot could trick the heuristic.  
  
Solution  
────────  
1.  New helper  _left_red_ratio()    
       – measures red pixels **only in the leftmost 35 %** of the box    
         (where the characteristic red square sits).  
  
2.  _is_energy_logo()  is tightened:    
       a)  global red ratio  >= MIN_RED_RATIO            (0.3 %)    
       b)  left red ratio    >= MIN_LEFT_RED_RATIO       (1.5 %)    
       c)  uniform background ring                       (unchanged)    
       d)  size / aspect / ink ratio                     (unchanged)  
  
3.  All other logic (two-box merge, logo-only page fallback,  
   seamless clone, header/footer safety…) is untouched.  
  
Result  
──────  
•   Body text (`… Hitachi Energy …`) is ignored.    
•   Screen-shots that merely contain the words are ignored.    
•   Genuine logos (with the red square) are still re-branded.    
  
Author:  Hitachi Energy – AI CoE  
"""  
  
from __future__ import annotations  
  
import io  
from pathlib import Path  
from typing import Dict, List, Tuple, Set  
  
import cv2  
import fitz  # PyMuPDF  
import numpy as np  
from PIL import Image, ImageDraw  
  
# ─────────────────────────────── USER SETTINGS ──────────────────────────────  
INPUT_DIR = "input_branding files"  
OUTPUT_DIR = "output_branding filesHE"  
  
AZURE_VISION_KEY = "0f365ad3f2c94a64ac9f92988cab8b09"  
AZURE_VISION_ENDPOINT = "https://cv-aicoe-poc-011b.cognitiveservices.azure.com/"  
  
REPLACEMENT_LOGOS: Dict[str, str] = {  
    "hitachi_inspire": "Hitachi_Inspire.png",  
    "hitachi_energy": "Hitachi_Energy2.png",  
}  
  
TEMPLATE_DIR = "templates"  
USE_TEMPLATE_DET = True  
DETECTION_BACKEND = "azure"  
DEBUG = False                     # True → dump PNG with detections  
  
# ───────────────────── TUNING CONSTANTS / HEURISTICS ────────────────────────  
LOGO_CONF_THRESHOLD = 0.55  
MIN_DIM, MAX_DIM = 50, 16_000     # Azure limits  
  
MIN_LOGO_HEIGHT_PX = 45  
HEADER_FOOTER_PCT = 0.22  
FOOTER_OCR_PCT = 0.25  
MIN_HITACHI_WIDTH_RATIO = 3.2  
IOU_DUP_THRESHOLD = 0.67  
  
MIN_RED_RATIO = 0.003             # ≥ 0.3 % red pixels  
MIN_LEFT_RED_RATIO = 0.015        # ≥ 1.5 % red pixels in left 35 %  
  
# screenshot / embedded-image filter  
EDGE_RING_PCT = 0.20  
MAX_EDGE_DENSITY = 0.06  
  
# “logo-only page” detector  
MIN_SOLO_INK_RATIO = 0.001  
MAX_SOLO_INK_RATIO = 0.08  
  
PAD_PCT, MAX_PAD_PX = 0.12, 12  
DILATE_PCT, MAX_DILATE_PX = 0.18, 15  
INPAINT_PCT, MAX_INPAINT_RADIUS = 0.07, 7  
  
SCALE_OVERSHOOT = 1.05  
SEAMLESS_CLONE_TRIES = (cv2.MIXED_CLONE, cv2.NORMAL_CLONE)  
  
KEYWORDS_HITACHI = ("hitachi", "inspire", "next", "energy")  
  
TEMPLATE_THRESHOLDS = {  
    "hitachi inspire the next": 0.63,  
    "hitachi energy": 0.55,  
}  
  
# ───────────────────────────────── HELPERS ──────────────────────────────────  
def ensure_dir(p: Path | str) -> None:  
    Path(p).mkdir(parents=True, exist_ok=True)  
  
  
def pil2bytes(img: Image.Image, fmt: str = "PNG") -> bytes:  
    b = io.BytesIO()  
    img.save(b, fmt)  
    return b.getvalue()  
  
  
def resize_for_azure(img: Image.Image) -> Tuple[Image.Image, float]:  
    w, h = img.size  
    if w < MIN_DIM or h < MIN_DIM:  
        s = float(MIN_DIM) / min(w, h)  
    elif w > MAX_DIM or h > MAX_DIM:  
        s = float(MAX_DIM) / max(w, h)  
    else:  
        return img, 1.0  
    return img.resize((int(w * s), int(h * s)), Image.LANCZOS), s  
  
  
# ────────────────────────── BOX / GEOM HELPERS ─────────────────────────────  
def _iou(a, b) -> float:  
    xa1, ya1, xa2, ya2 = a  
    xb1, yb1, xb2, yb2 = b  
    iw = max(0, min(xa2, xb2) - max(xa1, xb1))  
    ih = max(0, min(ya2, yb2) - max(ya1, yb1))  
    inter = iw * ih  
    if inter == 0:  
        return 0.0  
    return inter / ((xa2 - xa1) * (ya2 - ya1) + (xb2 - xb1) * (yb2 - yb1) - inter)  
  
  
def _expand_box(  
    box: Tuple[int, int, int, int], pad: int, max_w: int, max_h: int  
) -> Tuple[int, int, int, int]:  
    x1, y1, x2, y2 = box  
    return (  
        max(0, x1 - pad),  
        max(0, y1 - pad),  
        min(max_w, x2 + pad),  
        min(max_h, y2 + pad),  
    )  
  
  
# ───────────────────────────── I/O util ────────────────────────────────────  
def _load_logo(key: str) -> np.ndarray:  
    file = REPLACEMENT_LOGOS[key]  
    search = [Path(file), Path.cwd() / file]  
    if "__file__" in globals():  
        search.append(Path(__file__).parent / file)  
    for p in search:  
        if p.exists():  
            img = cv2.imread(str(p), cv2.IMREAD_UNCHANGED)  
            if img is not None:  
                return img  
    raise FileNotFoundError("Replacement logo not found:\n  " + "\n  ".join(map(str, search)))  
  
  
# ───────────────────────── DETECTION BACKENDS ──────────────────────────────  
def azure_detect(img: Image.Image) -> List[dict]:  
    """  
    Azure Vision v4 detector  
      • OBJECT     – full page  
      • READ-OCR   – full page (skip “Energy”)  
      • READ-OCR   – footer    (only “Energy”)  
      • READ-OCR   – full page (only “Energy”) – allows two-box merge  
    """  
    from azure.ai.vision.imageanalysis import ImageAnalysisClient  
    from azure.ai.vision.imageanalysis.models import VisualFeatures  
    from azure.core.credentials import AzureKeyCredential  
  
    cli = ImageAnalysisClient(  
        endpoint=AZURE_VISION_ENDPOINT,  
        credential=AzureKeyCredential(AZURE_VISION_KEY),  
    )  
  
    found: List[dict] = []  
  
    # OBJECT  
    obj = cli.analyze(  
        image_data=pil2bytes(img), visual_features=[VisualFeatures.OBJECTS]  
    )  
    if obj.objects:  
        for o in obj.objects.list:  
            tag = o.tags[0]  
            if float(tag.confidence or 0) < LOGO_CONF_THRESHOLD:  
                continue  
            if not any(k in tag.name.lower() for k in KEYWORDS_HITACHI):  
                continue  
            bb = o.bounding_box  
            found.append(  
                dict(  
                    label=tag.name,  
                    box=(bb.x, bb.y, bb.x + bb.w, bb.y + bb.h),  
                    source="object",  
                    conf=float(tag.confidence or 0),  
                )  
            )  
  
    # OCR helper  
    def _append(resp, y_shift, sc, want_energy):  
        if not resp.read or not getattr(resp.read, "blocks", None):  
            return  
        for blk in resp.read.blocks:  
            for ln in blk.lines:  
                txt = (ln.text or "").strip()  
                if not any(k in txt.lower() for k in KEYWORDS_HITACHI):  
                    continue  
                if ("energy" in txt.lower()) != want_energy:  
                    continue  
                xs = [p.x / sc for p in ln.bounding_polygon]  
                ys = [p.y / sc + y_shift for p in ln.bounding_polygon]  
                if xs:  
                    found.append(  
                        dict(  
                            label=txt,  
                            box=(min(xs), min(ys), max(xs), max(ys)),  
                            source="ocr",  
                            conf=1.0,  
                        )  
                    )  
  
    full_sc, sc_full = resize_for_azure(img)  
    ocr_full = cli.analyze(  
        image_data=pil2bytes(full_sc), visual_features=[VisualFeatures.READ]  
    )  
    _append(ocr_full, 0, sc_full, want_energy=False)  # skip “Energy”  
  
    # footer  (Energy only)  
    y0 = int(img.height * (1 - FOOTER_OCR_PCT))  
    foot = img.crop((0, y0, img.width, img.height))  
    foot_sc, sc_foot = resize_for_azure(foot)  
    ocr_foot = cli.analyze(  
        image_data=pil2bytes(foot_sc), visual_features=[VisualFeatures.READ]  
    )  
    _append(ocr_foot, y0, sc_foot, want_energy=True)  
  
    # full page  (Energy only)  
    _append(ocr_full, 0, sc_full, want_energy=True)  
  
    return found  
  
  
def template_detect(img: Image.Image) -> List[dict]:  
    if not USE_TEMPLATE_DET or not Path(TEMPLATE_DIR).exists():  
        return []  
    gray_full = cv2.cvtColor(np.asarray(img.convert("RGB")), cv2.COLOR_RGB2GRAY)  
    hits: List[dict] = []  
  
    for tpath in Path(TEMPLATE_DIR).glob("*"):  
        stem = tpath.stem.lower()  
        if "hitachi_inspire" in stem:  
            lbl = "hitachi inspire the next"  
        elif "hitachi_energy" in stem:  
            lbl = "hitachi energy"  
        else:  
            continue  
  
        tmpl = cv2.imread(str(tpath), cv2.IMREAD_GRAYSCALE)  
        if tmpl is None:  
            continue  
        th, tw = tmpl.shape  
        thr = TEMPLATE_THRESHOLDS.get(lbl, 0.63)  
  
        for sc in (1.0, 0.9, 0.8, 0.7, 0.6, 0.5):  
            gray = gray_full if sc == 1.0 else cv2.resize(  
                gray_full, None, fx=sc, fy=sc, interpolation=cv2.INTER_AREA  
            )  
            if gray.shape[0] < th or gray.shape[1] < tw:  
                continue  
            res = cv2.matchTemplate(gray, tmpl, cv2.TM_CCOEFF_NORMED)  
            ys, xs = np.where(res >= thr)  
            for y, x in zip(ys, xs):  
                hits.append(  
                    dict(  
                        label=lbl,  
                        source="tmpl",  
                        conf=1.0,  
                        box=(  
                            int(x / sc),  
                            int(y / sc),  
                            int((x + tw) / sc),  
                            int((y + th) / sc),  
                        ),  
                    )  
                )  
    return hits  
  
  
# ────────────── heuristics / filters ───────────────────────────────────────  
def _ink_ratio(img: Image.Image, box) -> float:  
    x1, y1, x2, y2 = map(int, box)  
    arr = np.asarray(img.crop((x1, y1, x2, y2)).convert("L"))  
    return (arr < 240).sum() / arr.size  
  
  
def _looks_like_logo(img: Image.Image, box) -> bool:  
    x1, y1, x2, y2 = map(int, box)  
    h, w = y2 - y1, x2 - x1  
    return h >= MIN_LOGO_HEIGHT_PX and w / h >= 2.0 and _ink_ratio(img, box) >= 0.07  
  
  
def _red_ratio(img: Image.Image, box) -> float:  
    x1, y1, x2, y2 = map(int, box)  
    hsv = cv2.cvtColor(np.asarray(img.crop((x1, y1, x2, y2))), cv2.COLOR_RGB2HSV)  
    mask = (cv2.inRange(hsv, (0, 100, 50), (10, 255, 255))  
            | cv2.inRange(hsv, (160, 100, 50), (180, 255, 255)))  
    return (mask > 0).sum() / mask.size  
  
  
def _left_red_ratio(img: Image.Image, box, pct: float = 0.35) -> float:  
    """Red pixel ratio in the leftmost *pct* of the bounding box."""  
    x1, y1, x2, y2 = map(int, box)  
    split = x1 + int((x2 - x1) * pct)  
    return _red_ratio(img, (x1, y1, split, y2))  
  
  
def _background_is_uniform(  
    img: Image.Image,  
    box,  
    ring_pct: float = EDGE_RING_PCT,  
    max_edge_density: float = MAX_EDGE_DENSITY,  
) -> bool:  
    x1, y1, x2, y2 = map(int, box)  
    w, h = x2 - x1, y2 - y1  
    pad = max(int(max(w, h) * ring_pct), 3)  
    X1, Y1, X2, Y2 = _expand_box(box, pad, img.width, img.height)  
  
    roi = cv2.cvtColor(np.asarray(img.crop((X1, Y1, X2, Y2))), cv2.COLOR_RGB2GRAY)  
    edges = cv2.Canny(roi, 100, 200)  
    edge_px = int(edges.sum() / 255)  
    ring_area = roi.size - w * h  
    if ring_area <= 0:  
        return True  
    return edge_px / ring_area < max_edge_density  
  
  
def _is_energy_logo(img: Image.Image, box) -> bool:  
    return (  
        _looks_like_logo(img, box)  
        and _red_ratio(img, box) >= MIN_RED_RATIO  
        and _left_red_ratio(img, box) >= MIN_LEFT_RED_RATIO  
        and _background_is_uniform(img, box)  
    )  
  
  
# ─── ensure red square is included (extend sideways) ───────────────────────  
def _extend_hitachi_energy_box(box, img, ratio: float = 1.2):  
    x1, y1, x2, y2 = map(int, box)  
    extra = int((y2 - y1) * ratio)  
    hsv = cv2.cvtColor(np.asarray(  
        img.crop((max(0, x1 - extra), y1, min(img.width, x2 + extra), y2))  
    ), cv2.COLOR_RGB2HSV)  
    mask = (cv2.inRange(hsv, (0, 100, 50), (10, 255, 255))  
            | cv2.inRange(hsv, (160, 100, 50), (180, 255, 255)))  
    left, right = mask[:, : int(mask.shape[1] * 0.3)].mean(), mask[:, -int(mask.shape[1] * 0.3):].mean()  
    if max(left, right) > 20:  
        if left > right:  
            x1 = max(0, x1 - extra)  
        else:  
            x2 = min(img.width, x2 + extra)  
    else:  
        x1 = max(0, x1 - extra)  
        x2 = min(img.width, x2 + extra)  
    return (x1, y1, x2, y2)  
  
  
# ────────────────────  “logo-only page”  detector  ─────────────────────────  
def detect_logo_only_page(img: Image.Image) -> List[dict]:  
    gray = cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2GRAY)  
    _, bw = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)  
    ink_ratio = np.count_nonzero(bw) / bw.size  
    if not (MIN_SOLO_INK_RATIO < ink_ratio < MAX_SOLO_INK_RATIO):  
        return []  
  
    bw = cv2.morphologyEx(bw, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))  
    cnts, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    if not cnts:  
        return []  
    x, y, w, h = cv2.boundingRect(max(cnts, key=cv2.contourArea))  
    box = (x, y, x + w, y + h)  
    if h < MIN_LOGO_HEIGHT_PX or w / h < 2.0:  
        return []  
    if not _is_energy_logo(img, box):  
        return []  
    return [{"label": "hitachi energy", "box": box}]  
  
  
# ───────────────────── merge / filter detections ───────────────────────────  
def final_logo_boxes(img: Image.Image) -> List[dict]:  
    sc_img, sc = resize_for_azure(img)  
  
    cand: List[dict] = []  
    if DETECTION_BACKEND == "azure":  
        cand.extend(azure_detect(sc_img))  
    cand.extend(template_detect(sc_img))  
  
    if sc != 1.0:  
        for c in cand:  
            x1, y1, x2, y2 = c["box"]  
            c["box"] = (int(x1 / sc), int(y1 / sc), int(x2 / sc), int(y2 / sc))  
  
    footer_y = int(img.height * (1 - FOOTER_OCR_PCT))  
  
    # keep ENERGY outside footer only if it looks logo-ish  
    cand = [  
        c  
        for c in cand  
        if not (  
            "energy" in c["label"].lower()  
            and (c["box"][1] + c["box"][3]) / 2 < footer_y  
            and not (_looks_like_logo(img, c["box"]) or _is_energy_logo(img, c["box"]))  
        )  
    ]  
  
    used: Set[int] = set()  
    final: List[dict] = []  
  
    def h_overlap(a, b):  
        return max(0, min(a[2], b[2]) - max(a[0], b[0]))  
  
    idx_hitachi = [i for i, c in enumerate(cand) if c["label"].lower() == "hitachi"]  
    idx_tagline = [  
        i for i, c in enumerate(cand)  
        if any(w in c["label"].lower() for w in ("inspire", "next"))  
    ]  
    idx_energy_word = [  
        i for i, c in enumerate(cand) if c["label"].strip().lower() == "energy"  
    ]  
  
    # Hitachi + Inspire  
    for hi in idx_hitachi:  
        if hi in used:  
            continue  
        hx1, hy1, hx2, hy2 = cand[hi]["box"]  
        hh = hy2 - hy1  
        union, grp = list(cand[hi]["box"]), [hi]  
        for ti in idx_tagline:  
            if ti in used:  
                continue  
            tx1, ty1, tx2, ty2 = cand[ti]["box"]  
            if not (hy1 <= ty1 <= hy2 + 3 * hh):  
                continue  
            if (h_overlap(cand[hi]["box"], cand[ti]["box"]) >= 0.3 * (tx2 - tx1)  
                    or hx1 <= (tx1 + tx2) / 2 <= hx2):  
                union[0] = min(union[0], tx1)  
                union[1] = min(union[1], ty1)  
                union[2] = max(union[2], tx2)  
                union[3] = max(union[3], ty2)  
                grp.append(ti)  
        if len(grp) == 1:  
            union[3] = min(img.height, union[3] + int(1.5 * hh))  
        if _looks_like_logo(img, union):  
            final.append({"label": "hitachi inspire the next", "box": tuple(map(int, union))})  
            used.update(grp)  
  
    # Hitachi + Energy (two-box) – footer  
    for hi in idx_hitachi:  
        if hi in used or cand[hi]["box"][1] < footer_y:  
            continue  
        hx1, hy1, hx2, hy2 = cand[hi]["box"]  
        hh, union, grp, merged = hy2 - hy1, list(cand[hi]["box"]), [hi], False  
        for ei in idx_energy_word:  
            if ei in used:  
                continue  
            ex1, ey1, ex2, ey2 = cand[ei]["box"]  
            if ey1 < footer_y or not (hy1 <= ey1 <= hy2 + 2.5 * hh):  
                continue  
            if (h_overlap(cand[hi]["box"], cand[ei]["box"]) >= 0.25 * (ex2 - ex1)  
                    or hx1 <= (ex1 + ex2) / 2 <= hx2):  
                union[0] = min(union[0], ex1)  
                union[1] = min(union[1], ey1)  
                union[2] = max(union[2], ex2)  
                union[3] = max(union[3], ey2)  
                grp.append(ei)  
                merged = True  
        if merged and _is_energy_logo(img, union):  
            final.append({"label": "hitachi energy", "box": tuple(map(int, union))})  
            used.update(grp)  
  
    # Hitachi + Energy (two-box) – centre/body  
    for hi in idx_hitachi:  
        if hi in used:  
            continue  
        hx1, hy1, hx2, hy2 = cand[hi]["box"]  
        if not (HEADER_FOOTER_PCT * img.height < hy1 < (1 - FOOTER_OCR_PCT) * img.height):  
            continue  
        hh, union, grp, merged = hy2 - hy1, list(cand[hi]["box"]), [hi], False  
        for ei in idx_energy_word:  
            if ei in used:  
                continue  
            ex1, ey1, ex2, ey2 = cand[ei]["box"]  
            if not (HEADER_FOOTER_PCT * img.height < ey1 < (1 - FOOTER_OCR_PCT) * img.height):  
                continue  
            if not (hy1 <= ey1 <= hy2 + 3 * hh):  
                continue  
            if (h_overlap(cand[hi]["box"], cand[ei]["box"]) >= 0.25 * (ex2 - ex1)  
                    or hx1 <= (ex1 + ex2) / 2 <= hx2):  
                union[0] = min(union[0], ex1)  
                union[1] = min(union[1], ey1)  
                union[2] = max(union[2], ex2)  
                union[3] = max(union[3], ey2)  
                grp.append(ei)  
                merged = True  
        if merged and _is_energy_logo(img, union):  
            final.append({"label": "hitachi energy", "box": tuple(map(int, union))})  
            used.update(grp)  
  
    # single-box  
    for i, c in enumerate(cand):  
        if i in used:  
            continue  
        if "hitachi" in c["label"].lower() and "energy" in c["label"].lower():  
            if _is_energy_logo(img, c["box"]):  
                final.append({"label": "hitachi energy", "box": tuple(map(int, c["box"]))})  
                used.add(i)  
  
    # template leftovers (non-energy)  
    for i, c in enumerate(cand):  
        if i in used or c["source"] != "tmpl" or "energy" in c["label"].lower():  
            continue  
        if _looks_like_logo(img, c["box"]):  
            final.append({"label": c["label"], "box": tuple(map(int, c["box"]))})  
            used.add(i)  
  
    # stand-alone “HITACHI”  
    for i, c in enumerate(cand):  
        if i in used or c["label"].lower() != "hitachi":  
            continue  
        x1, y1, x2, y2 = c["box"]  
        h, w = y2 - y1, x2 - x1  
        if h < MIN_LOGO_HEIGHT_PX or w / h < MIN_HITACHI_WIDTH_RATIO:  
            continue  
        if HEADER_FOOTER_PCT * img.height <= y2 <= (1 - HEADER_FOOTER_PCT) * img.height:  
            continue  
        union = (x1, y1, x2, min(img.height, y2 + int(1.5 * h)))  
        if _looks_like_logo(img, union):  
            final.append({"label": "hitachi inspire the next", "box": union})  
  
    # dedup  
    dedup: List[dict] = []  
    for m in final:  
        if not any(_iou(m["box"], d["box"]) > IOU_DUP_THRESHOLD for d in dedup):  
            dedup.append(m)  
    return dedup  
  
  
# ─────────────── α-BLEND (fallback) & helpers ──────────────────────────────  
def _alpha_overlay(dst: np.ndarray, src_rgba: np.ndarray, top: int, left: int):  
    top, left = int(top), int(left)  
    h, w = src_rgba.shape[:2]  
    if top < 0 or left < 0 or top + h > dst.shape[0] or left + w > dst.shape[1]:  
        return dst  
    roi = dst[top:top + h, left:left + w].astype(np.float32)  
    src_rgb = src_rgba[..., :3].astype(np.float32)  
  
    alpha = src_rgba[..., 3].astype(np.float32) / 255.0  
    alpha = cv2.GaussianBlur(alpha, (5, 5), 0)  
    if alpha.ndim == 2:  
        alpha = alpha[..., None]  
  
    dst[top:top + h, left:left + w] = (alpha * src_rgb + (1 - alpha) * roi).astype(np.uint8)  
    return dst  
  
  
def _prep_logo(key: str, tw: int, th: int) -> np.ndarray:  
    logo = _load_logo(key)  
    if (th > tw) != (logo.shape[0] > logo.shape[1]):  
        logo = cv2.rotate(logo, cv2.ROTATE_90_COUNTERCLOCKWISE)  
    sc = max(1e-6, min(tw / logo.shape[1], th / logo.shape[0]) * SCALE_OVERSHOOT)  
    return cv2.resize(  
        logo,  
        (max(1, int(logo.shape[1] * sc)), max(1, int(logo.shape[0] * sc))),  
        interpolation=cv2.INTER_LANCZOS4,  
    )  
  
  
# ───────────────────────────── W I P E + P A S T E ─────────────────────────  
def _kernel(side: int, pct: float, mx: int, odd=True) -> int:  
    k = max(3, min(mx, int(side * pct)))  
    return k + 1 if odd and k % 2 == 0 else k  
  
  
def _create_wipe_mask(cv: np.ndarray, box):  
    x1, y1, x2, y2 = map(int, box)  
    pad = _kernel(min(x2 - x1, y2 - y1), PAD_PCT, MAX_PAD_PX, odd=False)  
    wipe = _expand_box((x1, y1, x2, y2), pad, cv.shape[1], cv.shape[0])  
  
    mask = np.zeros(cv.shape[:2], np.uint8)  
    cv2.rectangle(mask, (wipe[0], wipe[1]), (wipe[2], wipe[3]), 255, -1)  
    dil = _kernel(min(x2 - x1, y2 - y1), DILATE_PCT, MAX_DILATE_PX)  
    mask = cv2.dilate(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (dil, dil)), 1)  
    rad = _kernel(min(x2 - x1, y2 - y1), INPAINT_PCT, MAX_INPAINT_RADIUS, odd=False)  
    return mask, wipe, rad  
  
  
def wipe_and_paste_generic(pil_img: Image.Image, box, key: str) -> Image.Image:  
    cv = cv2.cvtColor(np.asarray(pil_img), cv2.COLOR_RGB2BGR)  
    mask, wipe, rad = _create_wipe_mask(cv, box)  
    cv = cv2.inpaint(cv, mask, rad, cv2.INPAINT_TELEA)  
  
    x1, y1, x2, y2 = wipe  
    logo = _prep_logo(key, x2 - x1, y2 - y1)  
    lbgr, alpha = logo[..., :3], logo[..., 3]  
    if alpha.max() == 0:  
        g = cv2.cvtColor(lbgr, cv2.COLOR_BGR2GRAY)  
        alpha = cv2.threshold(g, 10, 255, cv2.THRESH_BINARY)[1]  
    cx, cy = x1 + (x2 - x1) // 2, y1 + (y2 - y1) // 2  
    ph, pw = cv.shape[:2]  
    cx = int(np.clip(cx, lbgr.shape[1] // 2, pw - (lbgr.shape[1] - lbgr.shape[1] // 2)))  
    cy = int(np.clip(cy, lbgr.shape[0] // 2, ph - (lbgr.shape[0] - lbgr.shape[0] // 2)))  
  
    blended = False  
    before = cv[y1:y2, x1:x2].copy()  
    for mode in SEAMLESS_CLONE_TRIES:  
        try:  
            trial = cv2.seamlessClone(lbgr, cv, alpha, (cx, cy), mode)  
            if np.mean(cv2.absdiff(trial[y1:y2, x1:x2], before)) > 5:  
                cv, blended = trial, True  
                break  
        except cv2.error:  
            pass  
    if not blended:  
        cv = _alpha_overlay(cv, logo, cy - lbgr.shape[0] // 2, cx - lbgr.shape[1] // 2)  
  
    return Image.fromarray(cv2.cvtColor(cv, cv2.COLOR_BGR2RGB))  
  
  
def wipe_and_paste(img, box):  
    return wipe_and_paste_generic(img, box, "hitachi_inspire")  
  
  
def wipe_and_paste_energy(img, box):  
    return wipe_and_paste_generic(img, box, "hitachi_energy")  
  
  
# ───────────────────────────── PDF WORKFLOW ────────────────────────────────  
def process_pdf(src: Path | str, dst: Path | str):  
    doc = fitz.open(src)  
    for pno, page in enumerate(doc):  
        pix = page.get_pixmap(dpi=300, alpha=False)  
        img = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)  
  
        detections = final_logo_boxes(img)  
  
        if not any("hitachi energy" in d["label"].lower() for d in detections):  
            detections.extend(detect_logo_only_page(img))  
  
        for d in detections:  
            if d["label"].lower() == "hitachi energy":  
                d["box"] = _extend_hitachi_energy_box(d["box"], img)  
  
        if DEBUG:  
            dbg = img.copy()  
            drw = ImageDraw.Draw(dbg)  
            for d in detections:  
                drw.rectangle(d["box"], outline="red", width=2)  
                drw.text((d["box"][0], d["box"][1] - 14), d["label"], fill="red")  
            dbg.save(f"DEBUG_page{pno+1}.png")  
  
        for d in detections:  
            if "energy" in d["label"].lower():  
                img = wipe_and_paste_energy(img, d["box"])  
            else:  
                img = wipe_and_paste(img, d["box"])  
  
        buf = io.BytesIO()  
        img.save(buf, "PNG")  
        page.clean_contents()  
        page.insert_image(page.rect, stream=buf.getvalue())  
  
    ensure_dir(Path(dst).parent)  
    doc.save(str(dst))  
    doc.close()  
  
  
def passthrough(src: Path | str, dst: Path | str):  
    Path(dst).write_bytes(Path(src).read_bytes())  
  
  
# ────────────────────────────────── MAIN ───────────────────────────────────  
def main():  
    ensure_dir(OUTPUT_DIR)  
    for fp in Path(INPUT_DIR).glob("*"):  
        out = Path(OUTPUT_DIR) / fp.name  
        if fp.suffix.lower() == ".pdf":  
            process_pdf(fp, out)  
        elif fp.suffix.lower() in (".docx", ".pptx"):  
            passthrough(fp, out)  
        else:  
            print("Skip", fp.name)  
        print("Finished file →", fp)  
    print("Finished →", OUTPUT_DIR)  
  
  
if __name__ == "__main__":  
    main()  

In [ ]:
#!/usr/bin/env python  
# -*- coding: utf-8 -*-  
"""  
══════════════════════════════════════════════════════════════════════════════  
                H I T A C H I   R E B R A N D I N G   T O O L  
══════════════════════════════════════════════════════════════════════════════  
  
Replaces ONLY genuine LOGOS  
    • “Hitachi Inspire the Next”  
    • “Hitachi Energy”  (incl. red square symbol)  
  
2025-05-26  –  “red-text” false-positive patch  
─────────────────────────────────────────────────────────────────  
Corner case  
───────────  
Body text that is printed IN RED (or red text placed directly in front  
of the words “Hitachi Energy”) fooled the former red-ratio filters.  
  
Fix  
───  
A new verifier `_has_left_red_square()` looks for one contiguous,  
almost-square red blob in the *left* half of the candidate.  The final  
logo decision `_is_energy_logo()` now requires:  
  
    1.  size / aspect / ink-ratio        (_looks_like_logo)  
    2.  calm background                  (_background_is_uniform)  
    3.  cheap colour pre-filters         (global + left red ratios)  
    4.  real red square present          (_has_left_red_square)  
  
All other logic (two-box merge, logo-only page fallback, screenshot  
protection, seamless clone, header/footer safety …) is unchanged.  
  
Author:  Hitachi Energy – AI CoE  
"""  
  
from __future__ import annotations  
  
import io  
from pathlib import Path  
from typing import Dict, List, Tuple, Set  
  
import cv2  
import fitz  # PyMuPDF  
import numpy as np  
from PIL import Image, ImageDraw  
  
# ─────────────────────────────── USER SETTINGS ──────────────────────────────  
INPUT_DIR = "input_branding files"  
OUTPUT_DIR = "output_branding filesHE"  
  
AZURE_VISION_KEY = "0f365ad3f2c94a64ac9f92988cab8b09"  
AZURE_VISION_ENDPOINT = "https://cv-aicoe-poc-011b.cognitiveservices.azure.com/"  
  
REPLACEMENT_LOGOS: Dict[str, str] = {  
    "hitachi_inspire": "Hitachi_Inspire.png",  
    "hitachi_energy": "Hitachi_Energy2.png",  
}  
  
TEMPLATE_DIR = "templates"  
USE_TEMPLATE_DET = True  
DETECTION_BACKEND = "azure"          # “azure” or “template”  
DEBUG = False                        # True → export debug PNGs  
  
# ───────────────────── TUNING CONSTANTS / HEURISTICS ────────────────────────  
LOGO_CONF_THRESHOLD = 0.55  
MIN_DIM, MAX_DIM = 50, 16_000                # Azure size limits  
MIN_LOGO_HEIGHT_PX = 45  
HEADER_FOOTER_PCT = 0.22  
FOOTER_OCR_PCT = 0.25  
MIN_HITACHI_WIDTH_RATIO = 3.2  
IOU_DUP_THRESHOLD = 0.67  
  
# colour / red-square heuristics  
MIN_RED_RATIO = 0.003            # ≥ 0.3 % red pixels  (cheap pre-filter)  
MIN_LEFT_RED_RATIO = 0.015       # ≥ 1.5 % red left   (cheap pre-filter)  
  
LEFT_HALF_PCT = 0.45             # search area for the square (≤ 45 % of width)  
MIN_RED_BLOCK_AREA_PCT = 0.04    # ≥ 4 % of logo area  
MIN_RED_BLOCK_AR = 0.6           # aspect-ratio of red square 0.6 – 1.4  
MAX_RED_BLOCK_AR = 1.4  
  
# screenshot / embedded-image filter  
EDGE_RING_PCT = 0.20  
MAX_EDGE_DENSITY = 0.06  
  
# “logo-only page” detector  
MIN_SOLO_INK_RATIO = 0.001  
MAX_SOLO_INK_RATIO = 0.08  
  
# wipe / in-paint  
PAD_PCT, MAX_PAD_PX = 0.12, 12  
DILATE_PCT, MAX_DILATE_PX = 0.18, 15  
INPAINT_PCT, MAX_INPAINT_RADIUS = 0.07, 7  
  
SCALE_OVERSHOOT = 1.05  
SEAMLESS_CLONE_TRIES = (cv2.MIXED_CLONE, cv2.NORMAL_CLONE)  
  
KEYWORDS_HITACHI = ("hitachi", "inspire", "next", "energy")  
  
TEMPLATE_THRESHOLDS = {  
    "hitachi inspire the next": 0.63,  
    "hitachi energy": 0.55,  
}  
  
# ───────────────────────────────── HELPERS ──────────────────────────────────  
def ensure_dir(p: Path | str) -> None:  
    Path(p).mkdir(parents=True, exist_ok=True)  
  
  
def pil2bytes(img: Image.Image, fmt: str = "PNG") -> bytes:  
    buf = io.BytesIO()  
    img.save(buf, fmt)  
    return buf.getvalue()  
  
  
def resize_for_azure(img: Image.Image) -> Tuple[Image.Image, float]:  
    w, h = img.size  
    if w < MIN_DIM or h < MIN_DIM:  
        s = float(MIN_DIM) / min(w, h)  
    elif w > MAX_DIM or h > MAX_DIM:  
        s = float(MAX_DIM) / max(w, h)  
    else:  
        return img, 1.0  
    return img.resize((int(w * s), int(h * s)), Image.LANCZOS), s  
  
  
# ────────────────────────── BOX / GEOM HELPERS ─────────────────────────────  
def _iou(a, b) -> float:  
    xa1, ya1, xa2, ya2 = a  
    xb1, yb1, xb2, yb2 = b  
    iw = max(0, min(xa2, xb2) - max(xa1, xb1))  
    ih = max(0, min(ya2, yb2) - max(ya1, yb1))  
    inter = iw * ih  
    if inter == 0:  
        return 0.0  
    return inter / ((xa2 - xa1) * (ya2 - ya1) +  
                    (xb2 - xb1) * (yb2 - yb1) - inter)  
  
  
def _expand_box(  
    box: Tuple[int, int, int, int], pad: int, max_w: int, max_h: int  
) -> Tuple[int, int, int, int]:  
    x1, y1, x2, y2 = box  
    return (  
        max(0, x1 - pad),  
        max(0, y1 - pad),  
        min(max_w, x2 + pad),  
        min(max_h, y2 + pad),  
    )  
  
# ───────────────────────────── I/O util ────────────────────────────────────  
def _load_logo(key: str) -> np.ndarray:  
    file = REPLACEMENT_LOGOS[key]  
    search = [Path(file), Path.cwd() / file]  
    if "__file__" in globals():  
        search.append(Path(__file__).parent / file)  
    for p in search:  
        if p.exists():  
            img = cv2.imread(str(p), cv2.IMREAD_UNCHANGED)  
            if img is not None:  
                return img  
    raise FileNotFoundError("Replacement logo not found:\n  " +  
                            "\n  ".join(map(str, search)))  
  
# ────────────── heuristics / filters ───────────────────────────────────────  
def _ink_ratio(img: Image.Image, box) -> float:  
    x1, y1, x2, y2 = map(int, box)  
    gray = np.asarray(img.crop((x1, y1, x2, y2)).convert("L"))  
    return (gray < 240).sum() / gray.size  
  
  
def _looks_like_logo(img: Image.Image, box) -> bool:  
    x1, y1, x2, y2 = map(int, box)  
    h, w = y2 - y1, x2 - x1  
    return h >= MIN_LOGO_HEIGHT_PX and w / h >= 2.0 and _ink_ratio(img, box) >= 0.07  
  
  
def _red_ratio(img: Image.Image, box) -> float:  
    x1, y1, x2, y2 = map(int, box)  
    hsv = cv2.cvtColor(np.asarray(img.crop((x1, y1, x2, y2))), cv2.COLOR_RGB2HSV)  
    mask = (cv2.inRange(hsv, (0, 100, 50), (10, 255, 255)) |  
            cv2.inRange(hsv, (160, 100, 50), (180, 255, 255)))  
    return mask.sum() / 255 / mask.size  
  
  
def _left_red_ratio(img: Image.Image, box, pct: float = 0.35) -> float:  
    x1, y1, x2, y2 = map(int, box)  
    split = x1 + int((x2 - x1) * pct)  
    return _red_ratio(img, (x1, y1, split, y2))  
  
  
def _background_is_uniform(  
    img: Image.Image,  
    box,  
    ring_pct: float = EDGE_RING_PCT,  
    max_edge_density: float = MAX_EDGE_DENSITY,  
) -> bool:  
    x1, y1, x2, y2 = map(int, box)  
    w, h = x2 - x1, y2 - y1  
    pad = max(int(max(w, h) * ring_pct), 3)  
    X1, Y1, X2, Y2 = _expand_box(box, pad, img.width, img.height)  
  
    roi = cv2.cvtColor(np.asarray(img.crop((X1, Y1, X2, Y2))), cv2.COLOR_RGB2GRAY)  
    edges = cv2.Canny(roi, 100, 200)  
    ring_area = roi.size - w * h  
    if ring_area <= 0:  
        return True  
    return edges.sum() / 255 / ring_area < max_edge_density  
  
  
# ─── NEW  red-square verifier ──────────────────────────────────────────────  
def _has_left_red_square(img: Image.Image, box) -> bool:  
    """  
    Detect *one* contiguous, almost-square red blob in the left part of `box`.  
    """  
    x1, y1, x2, y2 = map(int, box)  
    w, h = x2 - x1, y2 - y1  
    left_limit = x1 + int(w * LEFT_HALF_PCT)  
  
    hsv = cv2.cvtColor(np.asarray(img.crop((x1, y1, left_limit, y2))), cv2.COLOR_RGB2HSV)  
    mask = (cv2.inRange(hsv, (0, 100, 50), (10, 255, 255)) |  
            cv2.inRange(hsv, (160, 100, 50), (180, 255, 255)))  
  
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8))  
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    if not cnts:  
        return False  
    cnt = max(cnts, key=cv2.contourArea)  
    area = cv2.contourArea(cnt)  
    if area < MIN_RED_BLOCK_AREA_PCT * w * h:  
        return False  
    rx, ry, rw, rh = cv2.boundingRect(cnt)  
    ar = rw / rh if rh else 0  
    if not (MIN_RED_BLOCK_AR <= ar <= MAX_RED_BLOCK_AR):  
        return False  
    if rx + rw / 2 > w * LEFT_HALF_PCT:  
        return False  
    return True  
  
  
def _is_energy_logo(img: Image.Image, box) -> bool:  
    """  
    Final decision: cheap filters + uniform BG + real red square.  
    """  
    return (_looks_like_logo(img, box) and  
            _red_ratio(img, box) >= MIN_RED_RATIO and  
            _left_red_ratio(img, box) >= MIN_LEFT_RED_RATIO and  
            _background_is_uniform(img, box) and  
            _has_left_red_square(img, box))  
  
# ─── ensure red square is included for “Hitachi Energy” logos ──────────────  
def _extend_hitachi_energy_box(box, img, ratio: float = 1.2):  
    x1, y1, x2, y2 = map(int, box)  
    extra = int((y2 - y1) * ratio)  
    hsv = cv2.cvtColor(np.asarray(  
        img.crop((max(0, x1 - extra), y1, min(img.width, x2 + extra), y2))  
    ), cv2.COLOR_RGB2HSV)  
    mask = (cv2.inRange(hsv, (0, 100, 50), (10, 255, 255)) |  
            cv2.inRange(hsv, (160, 100, 50), (180, 255, 255)))  
    left = mask[:, : int(mask.shape[1] * 0.3)].mean()  
    right = mask[:, -int(mask.shape[1] * 0.3):].mean()  
    if max(left, right) > 20:  
        if left > right:  
            x1 = max(0, x1 - extra)  
        else:  
            x2 = min(img.width, x2 + extra)  
    else:  
        x1 = max(0, x1 - extra)  
        x2 = min(img.width, x2 + extra)  
    return (x1, y1, x2, y2)  
  
# ───────────────────────── DETECTION BACKENDS ──────────────────────────────  
def azure_detect(img: Image.Image) -> List[dict]:  
    """  
    Azure Vision v4 detector  
      • OBJECT     – full page  
      • READ-OCR   – full page  (skip Energy)  
      • READ-OCR   – footer     (Energy only)  
      • READ-OCR   – full page  (Energy only)   (for two-box merge)  
    """  
    from azure.ai.vision.imageanalysis import ImageAnalysisClient  
    from azure.ai.vision.imageanalysis.models import VisualFeatures  
    from azure.core.credentials import AzureKeyCredential  
  
    cli = ImageAnalysisClient(  
        endpoint=AZURE_VISION_ENDPOINT,  
        credential=AzureKeyCredential(AZURE_VISION_KEY),  
    )  
  
    found: List[dict] = []  
  
    # OBJECT detector  
    res_obj = cli.analyze(  
        image_data=pil2bytes(img),  
        visual_features=[VisualFeatures.OBJECTS],  
    )  
    if res_obj.objects:  
        for o in res_obj.objects.list:  
            tag = o.tags[0]  
            if float(tag.confidence or 0) < LOGO_CONF_THRESHOLD:  
                continue  
            if not any(k in tag.name.lower() for k in KEYWORDS_HITACHI):  
                continue  
            bb = o.bounding_box  
            found.append(  
                dict(  
                    label=tag.name,  
                    box=(bb.x, bb.y, bb.x + bb.w, bb.y + bb.h),  
                    conf=float(tag.confidence or 0),  
                    source="object",  
                )  
            )  
  
    # OCR helper  
    def _append(resp, y_shift, sc, want_energy):  
        if not resp.read or not getattr(resp.read, "blocks", None):  
            return  
        for blk in resp.read.blocks:  
            for ln in blk.lines:  
                txt = (ln.text or "").strip()  
                if not any(k in txt.lower() for k in KEYWORDS_HITACHI):  
                    continue  
                if ("energy" in txt.lower()) != want_energy:  
                    continue  
                xs = [p.x / sc for p in ln.bounding_polygon]  
                ys = [p.y / sc + y_shift for p in ln.bounding_polygon]  
                if xs:  
                    found.append(  
                        dict(  
                            label=txt,  
                            box=(min(xs), min(ys), max(xs), max(ys)),  
                            conf=1.0,  
                            source="ocr",  
                        )  
                    )  
  
    # full page (skip “Energy”)  
    full_sc, sc_full = resize_for_azure(img)  
    res_full = cli.analyze(  
        image_data=pil2bytes(full_sc), visual_features=[VisualFeatures.READ]  
    )  
    _append(res_full, 0, sc_full, want_energy=False)  
  
    # footer  (Energy only)  
    y0 = int(img.height * (1 - FOOTER_OCR_PCT))  
    foot = img.crop((0, y0, img.width, img.height))  
    foot_sc, sc_foot = resize_for_azure(foot)  
    res_foot = cli.analyze(  
        image_data=pil2bytes(foot_sc), visual_features=[VisualFeatures.READ]  
    )  
    _append(res_foot, y0, sc_foot, want_energy=True)  
  
    # full page  (Energy only)  
    _append(res_full, 0, sc_full, want_energy=True)  
  
    return found  
  
  
def template_detect(img: Image.Image) -> List[dict]:  
    if not USE_TEMPLATE_DET or not Path(TEMPLATE_DIR).exists():  
        return []  
    gray_full = cv2.cvtColor(np.asarray(img.convert("RGB")), cv2.COLOR_RGB2GRAY)  
    hits: List[dict] = []  
  
    for tpath in Path(TEMPLATE_DIR).glob("*"):  
        stem = tpath.stem.lower()  
        if "hitachi_inspire" in stem:  
            lbl = "hitachi inspire the next"  
        elif "hitachi_energy" in stem:  
            lbl = "hitachi energy"  
        else:  
            continue  
  
        tmpl = cv2.imread(str(tpath), cv2.IMREAD_GRAYSCALE)  
        if tmpl is None:  
            continue  
        th, tw = tmpl.shape  
        thr = TEMPLATE_THRESHOLDS.get(lbl, 0.63)  
  
        for sc in (1.0, 0.9, 0.8, 0.7, 0.6, 0.5):  
            gray = gray_full if sc == 1.0 else cv2.resize(  
                gray_full, None, fx=sc, fy=sc, interpolation=cv2.INTER_AREA  
            )  
            if gray.shape[0] < th or gray.shape[1] < tw:  
                continue  
            res = cv2.matchTemplate(gray, tmpl, cv2.TM_CCOEFF_NORMED)  
            ys, xs = np.where(res >= thr)  
            for y, x in zip(ys, xs):  
                hits.append(  
                    dict(  
                        label=lbl,  
                        source="tmpl",  
                        conf=1.0,  
                        box=(  
                            int(x / sc),  
                            int(y / sc),  
                            int((x + tw) / sc),  
                            int((y + th) / sc),  
                        ),  
                    )  
                )  
    return hits  
  
# ────────────────────  “logo-only page”  detector  ─────────────────────────  
def detect_logo_only_page(img: Image.Image) -> List[dict]:  
    gray = cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2GRAY)  
    _, bw = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)  
    ink_ratio = bw.sum() / 255 / bw.size  
    if not (MIN_SOLO_INK_RATIO < ink_ratio < MAX_SOLO_INK_RATIO):  
        return []  
  
    bw = cv2.morphologyEx(bw, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))  
    cnts, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    if not cnts:  
        return []  
    x, y, w, h = cv2.boundingRect(max(cnts, key=cv2.contourArea))  
    box = (x, y, x + w, y + h)  
    if h < MIN_LOGO_HEIGHT_PX or w / h < 2.0:  
        return []  
    if not _is_energy_logo(img, box):  
        return []  
    return [{"label": "hitachi energy", "box": box}]  
  
# ───────────────────── merge / filter detections ───────────────────────────  
def final_logo_boxes(img: Image.Image) -> List[dict]:  
    sc_img, sc = resize_for_azure(img)  
  
    cand: List[dict] = []  
    if DETECTION_BACKEND == "azure":  
        cand.extend(azure_detect(sc_img))  
    cand.extend(template_detect(sc_img))  
  
    if sc != 1.0:  
        for c in cand:  
            x1, y1, x2, y2 = c["box"]  
            c["box"] = (int(x1 / sc), int(y1 / sc), int(x2 / sc), int(y2 / sc))  
  
    footer_y = int(img.height * (1 - FOOTER_OCR_PCT))  
  
    # remove ENERGY words outside footer that clearly aren't logos  
    cand = [  
        c  
        for c in cand  
        if not (  
            "energy" in c["label"].lower()  
            and (c["box"][1] + c["box"][3]) / 2 < footer_y  
            and not (_looks_like_logo(img, c["box"]) or _is_energy_logo(img, c["box"]))  
        )  
    ]  
  
    used: Set[int] = set()  
    final: List[dict] = []  
  
    def h_overlap(a, b):  
        return max(0, min(a[2], b[2]) - max(a[0], b[0]))  
  
    idx_hitachi = [i for i, c in enumerate(cand) if c["label"].lower() == "hitachi"]  
    idx_tagline = [  
        i for i, c in enumerate(cand)  
        if any(w in c["label"].lower() for w in ("inspire", "next"))  
    ]  
    idx_energy_word = [  
        i for i, c in enumerate(cand)  
        if c["label"].strip().lower() == "energy"  
    ]  
  
    # Hitachi + Inspire  
    for hi in idx_hitachi:  
        if hi in used:  
            continue  
        hx1, hy1, hx2, hy2 = cand[hi]["box"]  
        hh = hy2 - hy1  
        union, grp = list(cand[hi]["box"]), [hi]  
        for ti in idx_tagline:  
            if ti in used:  
                continue  
            tx1, ty1, tx2, ty2 = cand[ti]["box"]  
            if not (hy1 <= ty1 <= hy2 + 3 * hh):  
                continue  
            if (h_overlap(cand[hi]["box"], cand[ti]["box"]) >= 0.3 * (tx2 - tx1)  
                    or hx1 <= (tx1 + tx2) / 2 <= hx2):  
                union[0] = min(union[0], tx1)  
                union[1] = min(union[1], ty1)  
                union[2] = max(union[2], tx2)  
                union[3] = max(union[3], ty2)  
                grp.append(ti)  
        if len(grp) == 1:  
            union[3] = min(img.height, union[3] + int(1.5 * hh))  
        if _looks_like_logo(img, union):  
            final.append({"label": "hitachi inspire the next",  
                          "box": tuple(map(int, union))})  
            used.update(grp)  
  
    # Hitachi + Energy (two-box) – footer  
    for hi in idx_hitachi:  
        if hi in used or cand[hi]["box"][1] < footer_y:  
            continue  
        hx1, hy1, hx2, hy2 = cand[hi]["box"]  
        hh = hy2 - hy1  
        union, grp, merged = list(cand[hi]["box"]), [hi], False  
        for ei in idx_energy_word:  
            if ei in used:  
                continue  
            ex1, ey1, ex2, ey2 = cand[ei]["box"]  
            if ey1 < footer_y or not (hy1 <= ey1 <= hy2 + 2.5 * hh):  
                continue  
            if (h_overlap(cand[hi]["box"], cand[ei]["box"]) >= 0.25 * (ex2 - ex1)  
                    or hx1 <= (ex1 + ex2) / 2 <= hx2):  
                union[0] = min(union[0], ex1)  
                union[1] = min(union[1], ey1)  
                union[2] = max(union[2], ex2)  
                union[3] = max(union[3], ey2)  
                grp.append(ei)  
                merged = True  
        if merged and _is_energy_logo(img, union):  
            final.append({"label": "hitachi energy",  
                          "box": tuple(map(int, union))})  
            used.update(grp)  
  
    # Hitachi + Energy (two-box) – centre/body  
    for hi in idx_hitachi:  
        if hi in used:  
            continue  
        hx1, hy1, hx2, hy2 = cand[hi]["box"]  
        if not (HEADER_FOOTER_PCT * img.height <  
                hy1 < (1 - FOOTER_OCR_PCT) * img.height):  
            continue  
        hh = hy2 - hy1  
        union, grp, merged = list(cand[hi]["box"]), [hi], False  
        for ei in idx_energy_word:  
            if ei in used:  
                continue  
            ex1, ey1, ex2, ey2 = cand[ei]["box"]  
            if not (HEADER_FOOTER_PCT * img.height <  
                    ey1 < (1 - FOOTER_OCR_PCT) * img.height):  
                continue  
            if not (hy1 <= ey1 <= hy2 + 3 * hh):  
                continue  
            if (h_overlap(cand[hi]["box"], cand[ei]["box"]) >= 0.25 * (ex2 - ex1)  
                    or hx1 <= (ex1 + ex2) / 2 <= hx2):  
                union[0] = min(union[0], ex1)  
                union[1] = min(union[1], ey1)  
                union[2] = max(union[2], ex2)  
                union[3] = max(union[3], ey2)  
                grp.append(ei)  
                merged = True  
        if merged and _is_energy_logo(img, union):  
            final.append({"label": "hitachi energy",  
                          "box": tuple(map(int, union))})  
            used.update(grp)  
  
    # single-box “Hitachi Energy”  
    for i, c in enumerate(cand):  
        if i in used:  
            continue  
        if "hitachi" in c["label"].lower() and "energy" in c["label"].lower():  
            if _is_energy_logo(img, c["box"]):  
                final.append({"label": "hitachi energy",  
                              "box": tuple(map(int, c["box"]))})  
                used.add(i)  
  
    # template leftovers (non-energy)  
    for i, c in enumerate(cand):  
        if i in used or c["source"] != "tmpl" or "energy" in c["label"].lower():  
            continue  
        if _looks_like_logo(img, c["box"]):  
            final.append({"label": c["label"],  
                          "box": tuple(map(int, c["box"]))})  
            used.add(i)  
  
    # stand-alone “HITACHI” in header/footer  
    for i, c in enumerate(cand):  
        if i in used or c["label"].lower() != "hitachi":  
            continue  
        x1, y1, x2, y2 = c["box"]  
        h, w = y2 - y1, x2 - x1  
        if h < MIN_LOGO_HEIGHT_PX or w / h < MIN_HITACHI_WIDTH_RATIO:  
            continue  
        if HEADER_FOOTER_PCT * img.height <= y2 <= (1 - HEADER_FOOTER_PCT) * img.height:  
            continue  
        union = (x1, y1, x2, min(img.height, y2 + int(1.5 * h)))  
        if _looks_like_logo(img, union):  
            final.append({"label": "hitachi inspire the next", "box": union})  
  
    # deduplicate  
    dedup: List[dict] = []  
    for m in final:  
        if not any(_iou(m["box"], d["box"]) > IOU_DUP_THRESHOLD for d in dedup):  
            dedup.append(m)  
    return dedup  
  
# ─────────────── α-BLEND (fallback) & helpers ──────────────────────────────  
def _alpha_overlay(dst: np.ndarray, src_rgba: np.ndarray, top: int, left: int):  
    top, left = int(top), int(left)  
    h, w = src_rgba.shape[:2]  
    if top < 0 or left < 0 or top + h > dst.shape[0] or left + w > dst.shape[1]:  
        return dst  
    roi = dst[top:top + h, left:left + w].astype(np.float32)  
    src_rgb = src_rgba[..., :3].astype(np.float32)  
  
    alpha = src_rgba[..., 3].astype(np.float32) / 255.0  
    alpha = cv2.GaussianBlur(alpha, (5, 5), 0)  
    if alpha.ndim == 2:  
        alpha = alpha[..., None]  
  
    dst[top:top + h, left:left + w] = (  
        alpha * src_rgb + (1 - alpha) * roi  
    ).astype(np.uint8)  
    return dst  
  
  
def _prep_logo(key: str, tw: int, th: int) -> np.ndarray:  
    logo = _load_logo(key)  
    if (th > tw) != (logo.shape[0] > logo.shape[1]):  
        logo = cv2.rotate(logo, cv2.ROTATE_90_COUNTERCLOCKWISE)  
    sc = max(1e-6, min(tw / logo.shape[1], th / logo.shape[0]) * SCALE_OVERSHOOT)  
    return cv2.resize(  
        logo,  
        (max(1, int(logo.shape[1] * sc)), max(1, int(logo.shape[0] * sc))),  
        interpolation=cv2.INTER_LANCZOS4,  
    )  
  
# ───────────────────────────── W I P E + P A S T E ─────────────────────────  
def _kernel(side: int, pct: float, mx: int, odd=True) -> int:  
    k = max(3, min(mx, int(side * pct)))  
    return k + 1 if odd and k % 2 == 0 else k  
  
  
def _create_wipe_mask(cv: np.ndarray, box):  
    x1, y1, x2, y2 = map(int, box)  
    pad = _kernel(min(x2 - x1, y2 - y1), PAD_PCT, MAX_PAD_PX, odd=False)  
    wipe = _expand_box((x1, y1, x2, y2), pad, cv.shape[1], cv.shape[0])  
  
    mask = np.zeros(cv.shape[:2], np.uint8)  
    cv2.rectangle(mask, (wipe[0], wipe[1]), (wipe[2], wipe[3]), 255, -1)  
    dil = _kernel(min(x2 - x1, y2 - y1), DILATE_PCT, MAX_DILATE_PX)  
    mask = cv2.dilate(mask,  
                      cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (dil, dil)),  
                      1)  
    rad = _kernel(min(x2 - x1, y2 - y1),  
                  INPAINT_PCT, MAX_INPAINT_RADIUS, odd=False)  
    return mask, wipe, rad  
  
  
def wipe_and_paste_generic(pil_img: Image.Image, box, key: str) -> Image.Image:  
    cv_img = cv2.cvtColor(np.asarray(pil_img), cv2.COLOR_RGB2BGR)  
    mask, wipe, rad = _create_wipe_mask(cv_img, box)  
    cv_img = cv2.inpaint(cv_img, mask, rad, cv2.INPAINT_TELEA)  
  
    x1, y1, x2, y2 = wipe  
    logo = _prep_logo(key, x2 - x1, y2 - y1)  
    lbgr, alpha = logo[..., :3], logo[..., 3]  
    if alpha.max() == 0:  
        g = cv2.cvtColor(lbgr, cv2.COLOR_BGR2GRAY)  
        alpha = cv2.threshold(g, 10, 255, cv2.THRESH_BINARY)[1]  
    cx, cy = x1 + (x2 - x1) // 2, y1 + (y2 - y1) // 2  
    ph, pw = cv_img.shape[:2]  
    cx = int(np.clip(cx, lbgr.shape[1] // 2,  
                     pw - (lbgr.shape[1] - lbgr.shape[1] // 2)))  
    cy = int(np.clip(cy, lbgr.shape[0] // 2,  
                     ph - (lbgr.shape[0] - lbgr.shape[0] // 2)))  
  
    blended = False  
    before = cv_img[y1:y2, x1:x2].copy()  
    for mode in SEAMLESS_CLONE_TRIES:  
        try:  
            trial = cv2.seamlessClone(lbgr, cv_img, alpha, (cx, cy), mode)  
            if np.mean(cv2.absdiff(trial[y1:y2, x1:x2], before)) > 5:  
                cv_img, blended = trial, True  
                break  
        except cv2.error:  
            pass  
    if not blended:  
        cv_img = _alpha_overlay(cv_img, logo,  
                                cy - lbgr.shape[0] // 2,  
                                cx - lbgr.shape[1] // 2)  
  
    return Image.fromarray(cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB))  
  
  
def wipe_and_paste(img, box):  
    return wipe_and_paste_generic(img, box, "hitachi_inspire")  
  
  
def wipe_and_paste_energy(img, box):  
    return wipe_and_paste_generic(img, box, "hitachi_energy")  
  
# ───────────────────────────── PDF WORKFLOW ────────────────────────────────  
def process_pdf(src: Path | str, dst: Path | str):  
    doc = fitz.open(src)  
    for pno, page in enumerate(doc):  
        pix = page.get_pixmap(dpi=300, alpha=False)  
        img = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)  
  
        detections = final_logo_boxes(img)  
  
        # fallback for pages that ONLY show a central HE logo  
        if not any("hitachi energy" in d["label"].lower() for d in detections):  
            detections.extend(detect_logo_only_page(img))  
  
        # extend HE box to include red square  
        for d in detections:  
            if d["label"].lower() == "hitachi energy":  
                d["box"] = _extend_hitachi_energy_box(d["box"], img)  
  
        if DEBUG:  
            dbg = img.copy()  
            drw = ImageDraw.Draw(dbg)  
            for d in detections:  
                drw.rectangle(d["box"], outline="red", width=2)  
                drw.text((d["box"][0], d["box"][1] - 14),  
                         d["label"], fill="red")  
            dbg.save(f"DEBUG_page{pno+1}.png")  
  
        # perform replacements  
        for d in detections:  
            if "energy" in d["label"].lower():  
                img = wipe_and_paste_energy(img, d["box"])  
            else:  
                img = wipe_and_paste(img, d["box"])  
  
        buf = io.BytesIO()  
        img.save(buf, "PNG")  
        page.clean_contents()  
        page.insert_image(page.rect, stream=buf.getvalue())  
  
    ensure_dir(Path(dst).parent)  
    doc.save(str(dst))  
    doc.close()  
  
# ─────────────────────────── passthrough for non-PDF ───────────────────────  
def passthrough(src: Path | str, dst: Path | str):  
    Path(dst).write_bytes(Path(src).read_bytes())  
  
# ────────────────────────────────── MAIN ───────────────────────────────────  
def main():  
    ensure_dir(OUTPUT_DIR)  
    for fp in Path(INPUT_DIR).glob("*"):  
        out = Path(OUTPUT_DIR) / fp.name  
        if fp.suffix.lower() == ".pdf":  
            process_pdf(fp, out)  
        elif fp.suffix.lower() in (".docx", ".pptx"):  
            passthrough(fp, out)  
        else:  
            print("Skip", fp.name)  
        print("Finished file →", fp)  
    print("Finished →", OUTPUT_DIR)  
  
  
if __name__ == "__main__":  
    main()  